In [1]:
import pandas as pd
import numpy as np

In [2]:
MIN_TRAIN_DAY = 1400

# 1. Load Data

In [3]:
sales_train_validation = pd.read_csv('raw_data\sales_train_validation.csv')
sample_submission = pd.read_csv('raw_data\sample_submission.csv')
sell_prices = pd.read_csv('raw_data\sell_prices.csv')
calendar = pd.read_csv('raw_data\calendar.csv')

In [4]:
sales_train_validation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [5]:
def process_calendar(calendar):
    calendar = calendar.drop('date',axis=1)
    calendar['day'] = calendar['d'].apply(lambda d : int(d[2:]))
    del calendar['d']
    return calendar
if 'd' in calendar.columns:
    calendar = process_calendar(calendar)

In [6]:
def get_one_day(df: pd.DataFrame, day, no_days):
    df_one_day = no_days.copy()
    df_one_day['day'] = day
    df_one_day['target'] = df[f'd_{str(day)}']
    return df_one_day
def multi_to_binary(df: pd.DataFrame, day_range = None):
    if day_range is None:
        day_range = [int(c[2:]) for c in df.columns if c.startswith('d_')]
    no_days = df[[c for c in df.columns if not c.startswith('d_')]]
    return pd.concat([get_one_day(df, day, no_days) for day in day_range])

def add_calendar(df, calendar):
#     df = df.join(calendar, on='day', rsuffix='_calendar')
#     df = df.drop('day_calendar', axis = 1)
    return pd.merge(df, calendar, how='left', on = ['day'], suffixes=('', '_calendar'))

def add_sell_prices(df, sell_prices):
    return pd.merge(df, sell_prices, how='left', on = ['item_id', 'store_id', 'wm_yr_wk'], suffixes=('', '_sellprices'))

def process_df(df,day_range = None):
    df = multi_to_binary(df, day_range)
    df['id'] = df['id'].apply(lambda s : s.replace('_validation','').replace('_evaluation',''))
    df = add_calendar(df, calendar)
    df = add_sell_prices(df, sell_prices)
    
    id_day_index = df[['id', 'day']].apply(lambda r: (r.id +'_'+str(r.day)), axis = 1)
    df = df.set_index(id_day_index)
    df = df[[c for c in df.columns if c != 'target'] + ['target']]
    return df

In [7]:
sample_submission_copy = sample_submission.copy()
splited_id = sample_submission_copy['id'].apply(lambda s: s.split('_'))
# item_id	dept_id	cat_id	store_id	state_id
sample_submission_copy['cat_id'] = splited_id.apply(lambda l: l[0])
sample_submission_copy['dept_id'] = splited_id.apply(lambda l: '_'.join(l[:2]))
sample_submission_copy['item_id'] = splited_id.apply(lambda l: '_'.join(l[:3]))
sample_submission_copy['state_id'] = splited_id.apply(lambda l: l[3])
sample_submission_copy['store_id'] = splited_id.apply(lambda l: '_'.join(l[3:5]))
is_private_test = splited_id.apply(lambda l: (l[-1] == 'evaluation'))
# remove the 'validation'
# sample_submission_copy['id'] = sample_submission_copy['id'].apply(lambda s : s.replace('_validation','').replace('_evaluation',''))
sample_submission_copy

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F24,F25,F26,F27,F28,cat_id,dept_id,item_id,state_id,store_id
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_001,CA,CA_1
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_002,CA,CA_1
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_003,CA,CA_1
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_004,CA,CA_1
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_005,CA,CA_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,FOODS,FOODS_3,FOODS_3_823,WI,WI_3
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,FOODS,FOODS_3,FOODS_3_824,WI,WI_3
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,FOODS,FOODS_3,FOODS_3_825,WI,WI_3
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,FOODS,FOODS_3,FOODS_3_826,WI,WI_3


In [8]:
public_submission = sample_submission_copy[~is_private_test]
private_submission = sample_submission_copy[is_private_test]

In [9]:
public_Fday_to_days = {f'F{str(c)}': f'd_{c + 1913}'for c in range(1, 30)}
private_Fday_to_days = {f'F{str(c)}': f'd_{c + 1941}'for c in range(1, 30)}
public_submission.rename(columns = public_Fday_to_days, inplace =True)
private_submission.rename(columns = private_Fday_to_days, inplace =True)

C:\Alon\Anaconda3\lib\site-packages\pandas\core\frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [10]:
public_submission

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1937,d_1938,d_1939,d_1940,d_1941,cat_id,dept_id,item_id,state_id,store_id
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_001,CA,CA_1
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_002,CA,CA_1
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_003,CA,CA_1
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_004,CA,CA_1
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,HOBBIES,HOBBIES_1,HOBBIES_1_005,CA,CA_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,FOODS,FOODS_3,FOODS_3_823,WI,WI_3
30486,FOODS_3_824_WI_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,FOODS,FOODS_3,FOODS_3_824,WI,WI_3
30487,FOODS_3_825_WI_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,FOODS,FOODS_3,FOODS_3_825,WI,WI_3
30488,FOODS_3_826_WI_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,FOODS,FOODS_3,FOODS_3_826,WI,WI_3


In [11]:
data = process_df(sales_train_validation, range(MIN_TRAIN_DAY, 1914))
public_submission = process_df(public_submission)
private_submission = process_df(private_submission)

In [12]:
data.sample(10)

,id,item_id,dept_id,cat_id,store_id,state_id,day,wm_yr_wk,weekday,wday,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,target
FOODS_3_703_CA_2_1797,FOODS_3_703_CA_2,FOODS_3_703,FOODS_3,FOODS,CA_2,CA,1797,11548,Wednesday,5,...,2015,NaN,NaN,NaN,NaN,0,0,0,1.98,1
HOUSEHOLD_1_218_CA_1_1591,HOUSEHOLD_1_218_CA_1,HOUSEHOLD_1_218,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1591,11519,Sunday,2,...,2015,NaN,NaN,NaN,NaN,1,1,0,1.94,2
HOBBIES_1_372_TX_1_1824,HOBBIES_1_372_TX_1,HOBBIES_1_372,HOBBIES_1,HOBBIES,TX_1,TX,1824,11552,Tuesday,4,...,2016,NaN,NaN,NaN,NaN,0,0,0,6.68,0
HOBBIES_1_325_TX_3_1587,HOBBIES_1_325_TX_3,HOBBIES_1_325,HOBBIES_1,HOBBIES,TX_3,TX,1587,11518,Wednesday,5,...,2015,NaN,NaN,NaN,NaN,1,1,1,6.44,1
HOUSEHOLD_2_189_CA_2_1604,HOUSEHOLD_2_189_CA_2,HOUSEHOLD_2_189,HOUSEHOLD_2,HOUSEHOLD,CA_2,CA,1604,11521,Saturday,1,...,2015,NaN,NaN,NaN,NaN,0,0,0,14.48,0
HOBBIES_1_240_CA_4_1813,HOBBIES_1_240_CA_4,HOBBIES_1_240,HOBBIES_1,HOBBIES,CA_4,CA,1813,11550,Friday,7,...,2016,NaN,NaN,NaN,NaN,0,1,1,2.98,0
FOODS_3_571_TX_3_1595,FOODS_3_571_TX_3,FOODS_3_571,FOODS_3,FOODS,TX_3,TX,1595,11519,Thursday,6,...,2015,NaN,NaN,NaN,NaN,0,1,1,8.98,1
FOODS_2_191_CA_3_1660,FOODS_2_191_CA_3,FOODS_2_191,FOODS_2,FOODS,CA_3,CA,1660,11529,Saturday,1,...,2015,NaN,NaN,NaN,NaN,0,1,1,8.88,0
FOODS_1_042_WI_3_1864,FOODS_1_042_WI_3,FOODS_1_042,FOODS_1,FOODS,WI_3,WI,1864,11606,Sunday,2,...,2016,NaN,NaN,NaN,NaN,1,1,1,0.94,1
HOBBIES_2_062_TX_1_1913,HOBBIES_2_062_TX_1,HOBBIES_2_062,HOBBIES_2,HOBBIES,TX_1,TX,1913,11613,Sunday,2,...,2016,NaN,NaN,NaN,NaN,0,0,0,5.88,0


In [13]:
data.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day',
       'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'sell_price', 'target'],
      dtype='object')

In [14]:
private_submission.columns

Index(['id', 'cat_id', 'dept_id', 'item_id', 'state_id', 'store_id', 'day',
       'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'sell_price', 'target'],
      dtype='object')

In [15]:
public_submission.columns

Index(['id', 'cat_id', 'dept_id', 'item_id', 'state_id', 'store_id', 'day',
       'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'sell_price', 'target'],
      dtype='object')

In [16]:
submission = pd.concat([public_submission, private_submission])

# 2. X, y split

In [17]:
X = data[[c for c in data.columns if c != 'target']]
y = data[['target']]

In [18]:
X

,id,item_id,dept_id,cat_id,store_id,state_id,day,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
HOBBIES_1_001_CA_1_1400,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1400,11443,Friday,7,11,2014,NaN,NaN,NaN,NaN,0,0,0,8.26
HOBBIES_1_002_CA_1_1400,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1400,11443,Friday,7,11,2014,NaN,NaN,NaN,NaN,0,0,0,3.97
HOBBIES_1_003_CA_1_1400,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1400,11443,Friday,7,11,2014,NaN,NaN,NaN,NaN,0,0,0,2.97
HOBBIES_1_004_CA_1_1400,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1400,11443,Friday,7,11,2014,NaN,NaN,NaN,NaN,0,0,0,4.64
HOBBIES_1_005_CA_1_1400,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1400,11443,Friday,7,11,2014,NaN,NaN,NaN,NaN,0,0,0,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_1913,FOODS_3_823_WI_3,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1913,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.98
FOODS_3_824_WI_3_1913,FOODS_3_824_WI_3,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1913,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.48
FOODS_3_825_WI_3_1913,FOODS_3_825_WI_3,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1913,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0,3.98
FOODS_3_826_WI_3_1913,FOODS_3_826_WI_3,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1913,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0,1.28


In [19]:
X_submission_public = public_submission[[c for c in public_submission.columns if c != 'target']]
y_submission_public_demo = public_submission['target']
X_submission_private = private_submission[[c for c in public_submission.columns if c != 'target']]
y_submission_private_demo = private_submission['target']
X_submission = submission[[c for c in submission.columns if c != 'target']]
y_submission_demo = submission['target']

In [ ]:
X.to_csv('data\X.csv')
y.to_csv('data\y.csv')
X_submission.to_csv('data\X_submission.csv')

# Submission

In [ ]:
# y_submission = y_submission_demo

In [ ]:
y_submission_public = y_submission_demo[X_submission['day'] <= X_public['day'].max()]
y_submission_private = y_submission_demo[X_submission['day'] > X_public['day'].max()]

In [ ]:
def process_y_submission(y_submission):
    y_submission = pd.DataFrame(y_submission)
    index_splitted = pd.Series(y_submission.index, index=y_submission.index).apply(lambda s: s.rsplit('_', 1))
    y_submission['new_index'] = index_splitted.apply(lambda l: l[0])
    y_submission['day'] = index_splitted.apply(lambda l: l[1])
    return y_submission

def parse_ss(s):
    ss = s.copy()
    day = ss['day'][0]
    ss = ss.set_index('new_index')
    ss = ss.drop('day', axis = 1)
    return ss, day

def process_submission(y_submission, index_suffix, Fday_to_days):
    y_submission = process_y_submission(y_submission)
    submission_df = pd.DataFrame(index=y_submission['new_index'].copy().unique())
    def add_to_submission(s):
        ss, day = parse_ss(s)
        submission_df[f'd_{day}'] = ss['target']
    y_submission.groupby('day').apply(add_to_submission)
    submission_df = submission_df.set_index(submission_df.index + index_suffix)
    submission_df = submission_df.rename(columns = {v:k for k,v in Fday_to_days.items()})
    return submission_df

In [ ]:
submission_public = process_submission(y_submission_public, '_validation', public_Fday_to_days)
submission_private = process_submission(y_submission_private, '_evaluation', private_Fday_to_days)

In [ ]:
submission = pd.concat([submission_public, submission_private])
submission = submission.reset_index().rename(columns = {'index': 'id'})

In [370]:
(submission['id'] == sample_submission['id']).all()

True

In [371]:
# submission.to_csv('submissions\submission.csv')

In [375]:
submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [373]:
sample_submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## A Few Statistics

In [23]:
uniq{col : sales_train_validation[col].unique() for col in sales_train_validation.columns}

{'id': array(['HOBBIES_1_001_CA_1_validation', 'HOBBIES_1_002_CA_1_validation',
        'HOBBIES_1_003_CA_1_validation', ...,
        'FOODS_3_825_WI_3_validation', 'FOODS_3_826_WI_3_validation',
        'FOODS_3_827_WI_3_validation'], dtype=object),
 'item_id': array(['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003', ...,
        'FOODS_3_825', 'FOODS_3_826', 'FOODS_3_827'], dtype=object),
 'dept_id': array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
        'FOODS_2', 'FOODS_3'], dtype=object),
 'cat_id': array(['HOBBIES', 'HOUSEHOLD', 'FOODS'], dtype=object),
 'store_id': array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
        'WI_2', 'WI_3'], dtype=object),
 'state_id': array(['CA', 'TX', 'WI'], dtype=object),
 'd_1': array([  0,  12,   2,   4,   5,   9,   3,   1,  25,  45,  10,   8,   7,
         16,  54,   6,  50,  14,  11,  17,  15,  23,  13,  20,  19,  38,
         32,  21,  30,  33, 107,  51,  26, 147,  22,  43,  24,  35,  27,
  